In [1]:
import re

In [2]:
with open ('../scripts/final_names.txt') as f:
    names = f.readlines()
    names = [x.strip() for x in names]

In [3]:
def es_tipo_witness(texto):
    out=False
    texto=quitar_tabla(texto)
    puntos=texto.find('WITNESS')
    if puntos != -1:
        out=True
    return out

def hay_statement_of(texto):
    out=False
    puntos=texto.find('STATEMENT OF')
    return puntos

def hay_tabla(texto):
    out=False
    puntos=texto.find('.....')
    if puntos != -1:
        bloque="".join(texto[:puntos].split(' '))
        if bloque.find('CONTENTS')!=-1:
            out=True
    return out
        
def mantener_tabla(texto):
    temp = texto
    out=""
    finished = False
    while not finished:
        definidor=temp.find('.....')
        if definidor!=-1:
            out += temp[:definidor+1]
            temp = temp[definidor+1:]
        else:
            finished = True
    return out

def quitar_tabla(texto):
    temp = texto
    finished = False
    while not finished:
        if temp.find('.....')!=-1:
            temp = temp[temp.find('.....')+1:]
        else:
            finished = True
    return temp

def hay_tabla_tipo1(texto):
    if hay_tabla(texto):
        tabla = mantener_tabla(texto)
        return tabla.lower().find("statement of witness") != -1
    return False

def hay_tabla_tipo2(texto):
    if hay_tabla(texto):
        tabla = mantener_tabla(texto)
        return tabla.find("WITNESS") != -1
    return False

In [4]:
def has_witnesses(texto):
    if hay_tabla_tipo1(texto) or hay_tabla_tipo2(texto):
        return True
    elif hay_statement_of(texto) != -1:
        indice = hay_statement_of(texto)
        paragraph = texto[indice-200:indice]
        return (paragraph.lower().find("witness") != -1) or (paragraph.lower().find("testimony") != -1)
    else:
        sin_tabla = quitar_tabla(texto)
        return sin_tabla.find("WITNESS") != -1

In [19]:
i = 0
j = 0
with open("witnesses_files.txt", 'a') as g:
    for filename in names[20000:]:
        with open("../data/{}".format(filename), encoding="ISO-8859-1") as f:
            text = f.read()
            if has_witnesses(text):
                print(filename, file=g)
                i += 1
        j += 1
        if j in [10, 100, 500, 1000, 2000, 3000, 4000]:
            print("Van {} documentos".format(j))

Van 10 documentos
Van 100 documentos
Van 500 documentos
Van 1000 documentos
Van 2000 documentos
Van 3000 documentos
Van 4000 documentos


In [20]:
print(i/len(names))

0.1420946782178218


<h3>Extracción de Witnesses</h3>

In [79]:
def get_witnesses_statements(texto, keyword):
    ans = []
    tmp_text = texto
    finished = False
    while not finished:
        indice = tmp_text.find(keyword)
        if indice != -1:
            lines = tmp_text[indice:].split("\n")
            witness = ""
            for l in lines:
                if re.search("[a-z]{4}", l) is None and len(l.strip()) > 0:
                    witness += " " + l.replace(keyword, "").strip()
                elif len(l.strip()) == 0:
                    continue
                else:
                    break
            ans.append(witness.strip())
            tmp_text = tmp_text[indice+1:]
        else:
            finished = True
    return ans

In [80]:
def get_witnesses(texto):
    ans = []
    clean_text = quitar_tabla(texto)
    ans = get_witnesses_statements(clean_text, "WITNESS") + get_witnesses_statements(clean_text, "WITNESSES")
    if len(ans) == 0:
        ans += get_witnesses_statements(clean_text, "STATEMENT OF ")
    return ans

In [81]:
with open("witnesses_files.txt") as f:
    filenames = [x.strip() for x in f.readlines()]

In [82]:
filename = filenames[14189]
with open("../data/{}".format(filename), encoding="ISO-8859-1") as f:
    text = f.read()
    print(get_witnesses(text))

['HON. RICHARD BURR, CHAIRMAN, A U.S. SENATOR FROM NORTH CAROLINA', 'HON. MARK R. WARNER, VICE CHAIRMAN, A U.S. SENATOR FROM VIRGINIA', 'ROY GODSON, Ph.D., EMERITUS PROFESSOR OF GOVERNMENT, GEORGETOWN UNIVERSITY', 'EUGENE B. RUMER, Ph.D., SENIOR FELLOW AND DIRECTOR, RUSSIA AND EURASIA PROGRAM, CARNEGIE ENDOWMENT FOR INTERNATIONAL PEACE', 'CLINT WATTS, ROBERT A. FOX FELLOW, FOREIGN POLICY RESEARCH INSTITUTE']


In [83]:
rows = []

In [ ]:
j = 0
for filename in filenames[:5000]:
    with open("../data/{}".format(filename), encoding="ISO-8859-1") as f:
        text = f.read()
        witnesses = get_witnesses(text)
        rows += [(filename, witness.split(",")[0], ", ".join(witness.split(",")[1:])) for witness in witnesses]
    j += 1
    if j in [10, 100, 500, 1000, 2000, 3000, 4000]:
        print("Van {} documentos".format(j))

Van 10 documentos
Van 100 documentos


In [54]:
import pandas as pd

In [55]:
df = pd.DataFrame(rows, columns=["filename", "name", "affiliation"])

In [75]:
df

,filename,name,affiliation
0,104th/House/Committee on Government Reform and...,ALAN KEMPER,FARMER
1,104th/House/Committee on Government Reform and...,BART DYE,FARMER
2,104th/House/Committee on Government Reform and...,WARREN BAIRD,FARMER
3,104th/House/Committee on Government Reform and...,JEAN ANN HARCOURT,"PRESIDENT, HARCOURT OUTLINES, INC."
4,104th/House/Committee on Government Reform and...,MALCOLM APPLEGATE,"PRESIDENT AND GENERAL MANAGER, INDIANAPOLIS ..."
5,104th/House/Committee on Government Reform and...,JEFF BOWE,"PRESIDENT, BENHAM PRESS"
6,104th/House/Committee on Government Reform and...,MYLES BRAND,"PRESIDENT, INDIANA UNIVERSITY"
7,104th/House/Committee on Government Reform and...,JOHN KEACH,"JR., PRESIDENT, HOME FEDERAL SAVINGS BANK"
8,104th/House/Committee on Government Reform and...,JERRY BAUMGARTNER,"PRESIDENT, TRI-COUNTY BANK AND TRUST"
9,104th/House/Committee on Government Reform and...,JEFF ROBINSON,"DIRECTOR OF WATER QUALITY, INDIANA AMERICAN ..."


In [78]:
df.iloc[74182]["filename"]

'115th/Senate/Select Committee on Intelligence/1.txt'

In [73]:
filename = filenames[14189]
with open("../data/{}".format(filename), encoding="ISO-8859-1") as f:
    text = f.read()

In [74]:
text

"\n - DISINFORMATION: A PRIMER IN RUSSIAN ACTIVE MEASURES AND INFLUENCE CAMPAIGNS PANEL I\n[Senate Hearing 115-40, Part 1]\n[From the U.S. Government Publishing Office]\n\n\n\n\n\n                                                  S. Hrg. 115-40, Pt. 1\n\n               DISINFORMATION: A PRIMER IN RUSSIAN ACTIVE\n                    MEASURES AND INFLUENCE CAMPAIGNS\n                                PANEL I\n\n=======================================================================\n\n                                HEARING\n\n                               BEFORE THE\n\n                    SELECT COMMITTEE ON INTELLIGENCE\n\n                                 OF THE\n\n                          UNITED STATES SENATE\n\n                     ONE HUNDRED FIFTEENTH CONGRESS\n\n                             FIRST SESSION\n\n                               __________\n\n                        THURSDAY, MARCH 30, 2017\n\n                               __________\n\n      Printed for the use of the S